In [ ]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y
!pip install '../input/library1/efficientnet_pytorch-0.7.1/dist/efficientnet_pytorch-0.7.1.tar'
!pip install '../input/library1/timm-0.4.12-py3-none-any.whl'

In [ ]:
import os
import efficientnet_pytorch
from PIL import Image
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
if df.shape[0] == 2477:
    fast_sub = True
    fast_df = pd.DataFrame(([['00086460a852_study', 'negative 1 0 0 1 1'], 
                         ['000c9c05fd14_study', 'negative 1 0 0 1 1'], 
                         ['65761e66de9f_image', 'none 1 0 0 1 1'], 
                         ['51759b5579bc_image', 'none 1 0 0 1 1']]), 
                       columns=['id', 'PredictionString'])
else:
    fast_sub = False

In [ ]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    # 近似3个标准差内数据，参考:http://www.kaggle.com/yukiszk/robust-pixel-array-scaling
    q1,q2,q3 = np.quantile(data, [.15865,.5,.84135]) 
    iqr = q3 - q1
    multiplier = 1.5
    mask = ((q2 - multiplier * iqr) < data) & (data < (q2 + multiplier * iqr))
    
    if data[mask].size != 0:
        p = .001
        data = data.astype(np.float32) - np.quantile(data[mask], p)
        data = data / np.quantile(data[mask], 1-p)
    else:
        data = data - np.min(data)
        data = data / np.max(data)

    data = np.clip(data, 0, 1)
    data = (data * 255).astype(np.uint8)
        
    return data

In [ ]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [ ]:
'''
split = 'test'
save_dir = f'/kaggle/tmp/{split}/'
os.makedirs(save_dir, exist_ok=True)

save_dir = f'/kaggle/tmp/{split}/study/'
os.makedirs(save_dir, exist_ok=True)
if fast_sub:
    xray = read_xray('/kaggle/input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
    im = resize(xray, size=600)  
    study = '00086460a852' + '_study.png'
    im.save(os.path.join(save_dir, study))
    xray = read_xray('/kaggle/input/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm')
    im = resize(xray, size=600)  
    study = '000c9c05fd14' + '_study.png'
    im.save(os.path.join(save_dir, study))
else:   
    for dirname, _, filenames in tqdm(os.walk(f'/kaggle/input/siim-covid19-detection/{split}')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size=600)  
            study = dirname.split('/')[-2] + '_study.png'
            im.save(os.path.join(save_dir, study))
'''

import time
import cv2
import torch
import torchvision
from skimage import io, transform,color
from glob import glob
from collections import Counter
from skimage import exposure
from torchvision import transforms
import torch.nn.functional as F
import torch
from torch.autograd import Variable
import albumentations as A

split = 'test'
save_dir = f'/kaggle/tmp/{split}/'
os.makedirs(save_dir, exist_ok=True)

img_size = 640
study, image, raw_h, raw_w = [], [], [], []
for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
    for file in filenames:
        # set keep_ratio=True to have original aspect ratio
        xray = read_xray(os.path.join(dirname, file))
        h, w = xray.shape
        #print(h,w)
        img = resize(xray, size=img_size)
        image_id = file.split('.')[0]
        study_id = dirname.split('/')[-2]

        image.append(image_id)
        study.append(study_id)
        raw_h.append(h)
        raw_w.append(w)
        img.save(f'{save_dir}/{image_id}.png')
        
data = {"ID": image, "StudyInstanceUID": study, "Raw H": raw_h, "Raw W": raw_w}
test_df = pd.DataFrame(data)

trans = A.Compose([A.Resize(512,512),
                   A.Normalize(mean=(0.485,0.456,0.406), std=(0.229, 0.224, 0.225))
])

# input data preprocess
def inference(img_path, input_size, model, mode='study'):
    ip_img = cv2.imread(img_path)
    ip_img = cv2.cvtColor(ip_img, cv2.COLOR_BGR2RGB)
    img_pre = trans(image=ip_img)
    img_pre = img_pre['image']
    to_tensor = transforms.ToTensor()
    input_tensor = to_tensor(img_pre)
    input_tensor = Variable(torch.unsqueeze(input_tensor, dim=0).float(), requires_grad=False).cuda()
    outputs = model(input_tensor)
    op_sfmax = F.softmax(outputs,dim=1).cpu().detach().numpy()[0]
    #print(op_sfmax)
    if mode == 'study':
        return np.argmax(op_sfmax),op_sfmax
        #return outputs
    else:
        pred = non_max_suppression(torch.tensor(outputs[0]), conf_thres = 0.15, iou_thres = 0.5)
        return pred
    

classes_map = {
    2: 'negative',
    3: 'typical',
    1: 'indeterminate',
    0: 'atypical',
}

#if __name__ == "__main__":

input_size_study = 512
input_size_img = 512

path = f'./../input/siim-covid19-detection/'
test_dir = f'/kaggle/tmp/test/'
sub_df = pd.read_csv(f'{path}/sample_submission.csv')
# init model
worker_study = torch.load(f'./../input/b7k5acc2/model_b7_ce_3.pth')
worker_study2 = torch.load(f'./../input/b7k5accc/model_b7_ce_acc_3.pth')
worker_study3 = torch.load(f'../input/b7k5accc/model_b7_ce_acc_1.pth')
worker_study4 = torch.load(f'../input/b7k5acc3/model_b7_ce_3.pth')
worker_study5 = torch.load(f'../input/b7k5accmulticlass/model_b7_mtclass_2.pth')
# worker_study = ONNXModel(f'./../input/efnetb0/efnet.onnx')
#worker_img = ONNXModel(f'./../input/models/yolov5m.onnx')

for index, row in sub_df.iterrows():
    #print(row['id'])
    mode = row['id'].split('_')[-1]
    if mode == 'study':
        study_id = row['id'].split('_')[0]
        image_paths = glob(f'{path}/test/{study_id}/*/*')
        pre_str = ''
        #for image_path in image_paths:
        image_path = image_paths[0]
        image_id = image_path.split('/')[-1].replace('.dcm', '')
        img_path = f'{test_dir}/{image_id}.png'
        img_cls,prob_list = inference(img_path, input_size_study, worker_study, mode='study')
        img_cls2,prob_list2 = inference(img_path, input_size_study, worker_study2, mode='study')
        img_cls3,prob_list3 = inference(img_path, input_size_study, worker_study3, mode='study')
        img_cls4,prob_list4 = inference(img_path, input_size_study, worker_study4, mode='study')
        img_cls5,prob_list5 = inference(img_path, input_size_study, worker_study5, mode='study')

        for i in classes_map:
             pre_str = pre_str + classes_map[i] + ' ' + str((prob_list[i]*0.22+prob_list2[i]*0.22+prob_list3[i]*0.22+prob_list4[i]*0.25+prob_list5[i]*0.09)) + ' 0 0 1 1 '

        sub_df.loc[index, "PredictionString"] = pre_str.rstrip()
    
df_study = sub_df

del worker_study, worker_study2,worker_study3,worker_study4,worker_study5

In [ ]:
image_id = []
dim0 = []
dim1 = []
splits = []
save_dir = f'/kaggle/tmp/{split}/image/'
os.makedirs(save_dir, exist_ok=True)
save_dir_512 = f'/kaggle/tmp/{split}/image512/'
os.makedirs(save_dir_512, exist_ok=True)
save_dir_1024 = f'/kaggle/tmp/{split}/image1024/'
os.makedirs(save_dir_1024, exist_ok=True)
if fast_sub:
    xray = read_xray('/kaggle/input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
    im = resize(xray, size=640)  
    im.save(os.path.join(save_dir,'65761e66de9f_image.png'))
    im_512 = resize(xray, size=512)  
    im_512.save(os.path.join(save_dir_512,'65761e66de9f_image.png'))
    im_1024 = resize(xray, size=1024)  
    im_1024.save(os.path.join(save_dir_1024,'65761e66de9f_image.png'))
    image_id.append('65761e66de9f.dcm'.replace('.dcm', ''))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
    splits.append(split)
    xray = read_xray('/kaggle/input/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm')
    im = resize(xray, size=640)  
    im.save(os.path.join(save_dir, '51759b5579bc_image.png'))
    im_512 = resize(xray, size=512)  
    im_512.save(os.path.join(save_dir_512, '51759b5579bc_image.png'))
    im_1024 = resize(xray, size=1024)  
    im_1024.save(os.path.join(save_dir_1024,'51759b5579bc_image.png'))
    image_id.append('51759b5579bc.dcm'.replace('.dcm', ''))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
    splits.append(split)
else:
    for dirname, _, filenames in tqdm(os.walk(f'/kaggle/input/siim-covid19-detection/{split}')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size=640)  
            im.save(os.path.join(save_dir, file.replace('.dcm', '_image.png')))
            im_512 = resize(xray, size=512) 
            im_512.save(os.path.join(save_dir_512, file.replace('.dcm', '_image.png')))
            im_1024 = resize(xray, size=1024) 
            im_1024.save(os.path.join(save_dir_1024, file.replace('.dcm', '_image.png')))
            image_id.append(file.replace('.dcm', ''))
            dim0.append(xray.shape[0])
            dim1.append(xray.shape[1])
            splits.append(split)
meta = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1, 'split': splits})

In [ ]:
import numpy as np 
import pandas as pd
if fast_sub:
    df = fast_df.copy()
else:
    df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
id_laststr_list  = []
for i in range(df.shape[0]):
    id_laststr_list.append(df.loc[i,'id'][-1])
df['id_last_str'] = id_laststr_list

study_len = df[df['id_last_str'] == 'y'].shape[0]

In [ ]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
import torch

In [ ]:
meta = meta[meta['split'] == 'test']
if fast_sub:
    test_df = fast_df.copy()
else:
    test_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
test_df = df[study_len:].reset_index(drop=True) 
meta['image_id'] = meta['image_id'] + '_image'
meta.columns = ['id', 'dim0', 'dim1', 'split']
test_df = pd.merge(test_df, meta, on = 'id', how = 'left')

In [ ]:
!pip install /kaggle/input/kerasapplications -q
!pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps

import os
import efficientnet.tfkeras as efn
import numpy as np
import pandas as pd
import tensorflow as tf

def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")

    return strategy


def build_decoder(with_labels=True, target_size=(300, 300), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img

    def decode_with_labels(path, label):
        return decode(path), label

    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img

    def augment_with_labels(img, label):
        return augment(img), label

    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)

    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)

    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)

    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)

    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)

    return dset


In [ ]:
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 16

IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600, 512)

'''
if fast_sub:
    sub_df = fast_df.copy()
else:
    sub_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
sub_df = sub_df[:study_len]
test_paths = f'/kaggle/tmp/{split}/study/' + sub_df['id'] +'.png'

sub_df['negative'] = 0
sub_df['typical'] = 0
sub_df['indeterminate'] = 0
sub_df['atypical'] = 0

label_cols = sub_df.columns[2:]

test_decoder = build_decoder(with_labels=False, target_size=(IMSIZE[7], IMSIZE[7]), ext='png')
dtest = build_dataset(
    test_paths, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=False, cache=False,
    decode_fn=test_decoder
)

with strategy.scope():
    
    models = []
    
    models0 = tf.keras.models.load_model(
        '../input/model00/model0.h5'
    )
    models1 = tf.keras.models.load_model(
        '../input/model00/model1.h5'
    )
    models2 = tf.keras.models.load_model(
        '../input/model00/model2.h5'
    )
    models3 = tf.keras.models.load_model(
        '../input/model00/model3.h5'
    )
    models4 = tf.keras.models.load_model(
        '../input/model00/model4.h5'
    )
    models5 = tf.keras.models.load_model(
        '../input/k/h053473666/siim-covid19-efnb7-train-study/model0.h5'
    )
    models6 = tf.keras.models.load_model(
        '../input/k/h053473666/siim-covid19-efnb7-train-study/model1.h5'
    )
    models7 = tf.keras.models.load_model(
        '../input/k/h053473666/siim-covid19-efnb7-train-study/model2.h5'
    )
    models8 = tf.keras.models.load_model(
        '../input/k/h053473666/siim-covid19-efnb7-train-study/model3.h5'
    )
    models9 = tf.keras.models.load_model(
        '../input/k/h053473666/siim-covid19-efnb7-train-study/model4.h5'
    )
#     models5 = tf.keras.models.load_model(
#         '../input/siim-covid19-efnb7-train-study/model0.h5'
#     )
#     models6 = tf.keras.models.load_model(
#         '../input/siim-covid19-efnb7-train-study/model1.h5'
#     )
#     models7 = tf.keras.models.load_model(
#         '../input/siim-covid19-efnb7-train-study/model2.h5'
#     )
#     models8 = tf.keras.models.load_model(
#         '../input/siim-covid19-efnb7-train-study/model3.h5'
#     )
#     models9 = tf.keras.models.load_model(
#         '../input/siim-covid19-efnb7-train-study/model4.h5'
#     )

    models.append(models0)
    models.append(models1)
    models.append(models2)
    models.append(models3)
    models.append(models4)
    models.append(models5)
    models.append(models6)
    models.append(models7)
    models.append(models8)
    models.append(models9)

    
sub_df[label_cols] = sum([model.predict(dtest, verbose=1) for model in models]) / len(models)
'''

In [ ]:
'''
sub_df.columns = ['id', 'PredictionString1', 'negative', 'typical', 'indeterminate', 'atypical']
df = pd.merge(df, sub_df, on = 'id', how = 'left')
for i in range(study_len):
    negative = df.loc[i,'negative']
    typical = df.loc[i,'typical']
    indeterminate = df.loc[i,'indeterminate']
    atypical = df.loc[i,'atypical']
    df.loc[i, 'PredictionString'] = f'negative {negative} 0 0 1 1 typical {typical} 0 0 1 1 indeterminate {indeterminate} 0 0 1 1 atypical {atypical} 0 0 1 1'

df_study = df[['id', 'PredictionString']]
'''

In [ ]:
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 16

IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600, 512)
if fast_sub:
    sub_df = fast_df.copy()
else:
    sub_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
sub_df = sub_df[study_len:]
test_paths = f'/kaggle/tmp/{split}/image512/' + sub_df['id'] +'.png'
sub_df['none'] = 0

label_cols = sub_df.columns[2]

test_decoder = build_decoder(with_labels=False, target_size=(IMSIZE[8], IMSIZE[8]), ext='png')
dtest = build_dataset(
    test_paths, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=False, cache=False,
    decode_fn=test_decoder
)

with strategy.scope():
    
    models = []
    
    models0 = tf.keras.models.load_model(
        '../input/siim-covid19-efnb7-train-2class-image/model0.h5'
    )
    models1 = tf.keras.models.load_model(
        '../input/siim-covid19-efnb7-train-2class-image/model1.h5'
    )
    models2 = tf.keras.models.load_model(
        '../input/siim-covid19-efnb7-train-2class-image/model2.h5'
    )
    models3 = tf.keras.models.load_model(
        '../input/siim-covid19-efnb7-train-2class-image/model3.h5'
    )
    models4 = tf.keras.models.load_model(
        '../input/siim-covid19-efnb7-train-2class-image/model4.h5'
    )
    
    models.append(models0)
    models.append(models1)
    models.append(models2)
    models.append(models3)
    models.append(models4)

    
sub_df[label_cols] = sum([model.predict(dtest, verbose=1) for model in models]) / len(models)
df_2class = sub_df.reset_index(drop=True)

In [ ]:
del models
del models0, models1, models2, models3, models4

In [ ]:
# 切换至GPU
from numba import cuda
import torch
#cuda.select_device(0)
#cuda.close()
#cuda.select_device(0)

In [ ]:
dim = 640 #1024, 256, 'original'
test_dir = f'/kaggle/tmp/{split}/image'

shutil.copytree('/kaggle/input/d/xuxiaoxi/yolov5min/yolo5-640', '/kaggle/working/yolo5-640')
os.chdir('/kaggle/working/yolo5-640') # install dependencies

In [ ]:
from common import *
from configure import *

from yolo5.models.yolo import *
from yolo5.utils.torch_utils import *
from yolo5.utils.general import bbox_iou, xywh2xyxy

class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()

        model_cfg_file = 'yolo5/models/yolov5x.yaml'
        pretrain_file  = '/kaggle/input/ultralyticsyolov5aweights/yolov5x.pt'
        e = Model(
            cfg=model_cfg_file,
            ch=3,
            nc=1,
            anchors=np.array(anchor_size).reshape(num_head,-1).tolist(),
        )
        state_dict = torch.load(pretrain_file, map_location=lambda storage, loc: storage)['model'].float().state_dict()
        state_dict = intersect_dicts(state_dict, e.state_dict(), exclude=['anchor'])  # intersect
        e.load_state_dict(state_dict, strict=False)

        #---
        # remove detect layer
        assert (e.save[-3:] == e.model[-1].f)
        removed = list(e.model.children())[:-1]
        self.backbone = torch.nn.Sequential(*removed)
        self.index = e.save
        #print(self.index)

        #---
        self.head = nn.ModuleList([
            nn.Conv2d(320, num_anchor*6, kernel_size=1),
            nn.Conv2d(640, num_anchor*6, kernel_size=1),
            nn.Conv2d(1280, num_anchor*6, kernel_size=1),
        ])

        #---
        #<todo> add image classification head
        # self.index.append( ... add feature layer no to use ...)
        self.logit=nn.Sequential(
            nn.Conv2d(1280, 256, kernel_size=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, kernel_size=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(128, 2, kernel_size=1),
        )


    def forward(self, image):
        batch_size = len(image)
        x = 2*image-1

        # yolov5 backbone ----------------------
        # predict = self.e(x)
        z = []
        for m in self.backbone:
            if m.f != -1:  # if not from previous layer
                if isinstance(m.f, int):
                    x = z[m.f]
                else:
                    x = [x if i == -1 else z[i] for i in m.f]
            x = m(x)  # run
            z.append(x if m.i in self.index else None)  # cache output
        z = [z[i] for i in self.index[-3:]]
        #--------------------------------------
        predict = []
        for n in range(num_head):
            p = self.head[n](z[n])
            batch_size, num_anchor_dim, h, w = p.shape
            dim = num_anchor_dim//num_anchor
            p = p.reshape(batch_size, num_anchor, dim, h, w).permute(0, 1, 3, 4, 2).contiguous()
            predict.append(p)
        logit = self.logit(z[-1]).reshape(batch_size, 2)

        return predict, logit


In [ ]:
device = torch.device("cuda")
print(device)

In [ ]:
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from model import *

def scale_coords(image_height, image_width, bboxes):
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]  * image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]  * image_height
    
    return bboxes

def data_process(img_path, input_size):
    img_raw = cv2.imread(img_path)
    img_resize = cv2.resize(img_raw, (input_size, input_size))
    img = img_resize.astype(np.float32)
    img /= 255.0
    img = img.transpose(2, 0, 1).astype(np.float32)
    input_tensor = torch.from_numpy(np.expand_dims(img, axis=0))
    return input_tensor
    
def softmax(x):
    row_max = np.max(x, axis=1).reshape(-1,1)
    x -= row_max
    x_exp = np.exp(x)
    return x_exp / np.sum(x_exp, axis=1, keepdims=True)

def do_predict(net, input_tensor, tta=[]):
    net.eval()
    with torch.no_grad():
        predict, logit = net(input_tensor)
        predict = infer_prediction(predict)
        predict_flat = pyramid_to_flat(predict)
        
        detection = do_non_max_suppression(
            predict_flat,
            nms_objectness_threshold=0.0001,
            nms_iou_threshold = 0.4,
            nms_pre_max_num=500,
            nms_post_max_num=25 
        )
    return detection, softmax(logit.cpu().numpy())

boxes_list = []
scores_list = []
labels_list = []

weights_dir = ['/kaggle/input/models-yolo-f0/f0_00021000_model.pth',
    '/kaggle/input/models-yolo-f1/f1_00020500_model.pth',
    '/kaggle/input/models-yolo-f2/f2_00019000_model.pth',
    '/kaggle/input/models-yolo-f3/f3_00019500_model.pth',
    '/kaggle/input/models-yolo-f4/f4_00018000_model.pth'  
]
device = torch.device("cuda")
for flod in [0,1,2,3,4]:
    
    net = Net1().cuda()
    #net= net.to(device)
    net.load_state_dict(torch.load(weights_dir[flod])['state_dict'], strict=True)
    
    b_list = []
    s_list = []
    l_list = []
    for img in os.listdir(test_dir):
        img_id = img.replace('.png', '')
        img_path = f'{test_dir}/{img}'
        input_tensor = data_process(img_path, dim).cuda()
    
        detection, img_cls = do_predict(net, input_tensor)
        bboxes = (detection[0][:, :4] / 640).tolist()
        scores = detection[0][:, 4].tolist()
        b_list.append(bboxes)
        s_list.append(scores)
        l_list.append([0] * len(bboxes))
    boxes_list.append(b_list)
    scores_list.append(s_list)
    labels_list.append(l_list)

In [ ]:
!pip install /kaggle/input/mmdetectionv2130/src/addict-2.4.0-py3-none-any.whl
!pip install /kaggle/input/mmdetectionv2130/src/yapf-0.31.0-py2.py3-none-any.whl
!pip install /kaggle/input/mmdetectionv2130/src/mmcv_full-1.3.7-cp37-cp37m-manylinux1_x86_64.whl
!rsync -a /kaggle/input/mmdetectionv2130/mmdetection /kaggle/
!pip install /kaggle/input/mmdetectionv2130/src/mmpycocotools-12.0.3
!pip install /kaggle/input/pycocotools202/pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl
!pip install /kaggle/input/ensemble-boxes/ensemble_boxes-1.0.6-py3-none-any.whl
!cd /kaggle/mmdetection && pip install -e .

In [ ]:
import sys
sys.path.insert(0, "/kaggle/mmdetection")
import cv2
from pathlib import Path
from ensemble_boxes import *
import mmcv
from mmcv import Config
from mmdet.models import build_detector
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel
from mmdet.datasets import build_dataloader, build_dataset
from mmdet.apis import single_gpu_test
from mmdet.apis import init_detector, inference_detector, show_result_pyplot

In [ ]:
mmdet_models = ['retinanet', 'vfnet', 'faster_rcnn', 'cascade_rcnn', 'retinanet1024', 'vfnet1024']
mmdet_models_1024 = ['retinanet1024','vfnet1024']
models_config = {'retinanet': "/kaggle/mmdetection/configs/retinanet/retinanet_r101_fpn_1x_coco.py",
                 'vfnet': "/kaggle/mmdetection/configs/vfnet/vfnet_r101_fpn_1x_coco.py", 
                 'faster_rcnn': "/kaggle/mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py", 
                 'cascade_rcnn': "/kaggle/mmdetection/configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco.py",
                 'retinanet1024': "/kaggle/mmdetection/configs/retinanet/retinanet_r101_fpn_1x_coco.py", 
                 'vfnet1024': "/kaggle/mmdetection/configs/vfnet/vfnet_r101_fpn_1x_coco.py"
                }

models_checkpoint = {'retinanet': "/kaggle/input/mmdetmodel0707/retinanet_r101.pth",
                 'vfnet': "/kaggle/input/models-mmdet-v1/vfnet_r101.pth", 
                 'faster_rcnn': "/kaggle/input/mmdetmodels/faster_rcnn50.pth", 
                 'cascade_rcnn': "/kaggle/input/mmdetmodels/cascade_rcnn50.pth",
                 'retinanet1024': "/kaggle/input/mmdet-model1024-0714/retinanet_1024.pth",
                 'vfnet1024': "/kaggle/input/mmdet-model1024-0714/vfnet_1024.pth"
                }
batch_size = 1
score_threshold = 0.0001
iou_threshold = 0.4
for mmdet_model in mmdet_models:
    baseline_cfg_path = models_config[mmdet_model]
    cfg = Config.fromfile(baseline_cfg_path)
    cfg.data.samples_per_gpu = batch_size # Batch size of a single GPU used in testing
    cfg.data.workers_per_gpu = 2 # Worker to pre-fetch data for each single GPU
    cfg.test_pipeline = [
        dict(type='LoadImageFromFile'),
        dict(
            type='MultiScaleFlipAug',
            img_scale=(640, 640),
            flip=False,
            transforms=[
                dict(type='Resize', keep_ratio=True),
                dict(type='RandomFlip'),
                dict(
                    type='Normalize',
                    mean=[123.675, 116.28, 103.53],
                    std=[58.395, 57.12, 57.375],
                    to_rgb=True),
                dict(type='Pad', size_divisor=32),
                dict(type='ImageToTensor', keys=['img']),
                dict(type='Collect', keys=['img'])
            ])
    ]
    if mmdet_model == 'retinanet':
        cfg.model.bbox_head.num_classes = 2 # retinanet vfnet
        cfg.model.bbox_head.anchor_generator.ratios = [0.25, 0.5, 1.0, 2.0, 4.0] # retinanet
        cfg.model.test_cfg.score_thr=score_threshold
        cfg.model.test_cfg.nms.iou_threshold=iou_threshold
        
    elif mmdet_model == 'vfnet':
        cfg.model.bbox_head.num_classes = 2 # retinanet vfnet
        cfg.model.test_cfg.score_thr=score_threshold
        cfg.model.test_cfg.nms.iou_threshold=iou_threshold
        
    elif mmdet_model == 'faster_rcnn':
        cfg.model.roi_head.bbox_head.num_classes = 2 # faster rcnn
        cfg.model.rpn_head.anchor_generator.ratios = [0.25, 0.5, 1.0, 2.0, 4.0] # rcnn
        cfg.model.test_cfg.rcnn.score_thr=score_threshold # rcnn
        cfg.model.test_cfg.rcnn.nms.iou_threshold=iou_threshold # rcnn
        
    elif mmdet_model == 'cascade_rcnn':
        cfg.model.roi_head.bbox_head[0].num_classes = 2 # cascade rcnn
        cfg.model.roi_head.bbox_head[1].num_classes = 2
        cfg.model.roi_head.bbox_head[2].num_classes = 2
        cfg.model.rpn_head.anchor_generator.ratios = [0.25, 0.5, 1.0, 2.0, 4.0] # rcnn
        cfg.model.test_cfg.rcnn.score_thr=score_threshold # rcnn
        cfg.model.test_cfg.rcnn.nms.iou_threshold=iou_threshold # rcnn
    elif mmdet_model == 'retinanet1024':
        cfg.model.bbox_head.num_classes = 2 # retinanet vfnet
        cfg.model.bbox_head.anchor_generator.ratios = [0.25, 0.5, 1.0, 2.0, 4.0] # retinanet
        cfg.model.test_cfg.score_thr=score_threshold
        cfg.model.test_cfg.nms.iou_threshold=iou_threshold
        cfg.test_pipeline[1]['img_scale'] = (1024, 1024)
        
    elif mmdet_model == 'vfnet1024':
        cfg.model.bbox_head.num_classes = 2 # retinanet vfnet
        cfg.model.test_cfg.score_thr=score_threshold
        cfg.model.test_cfg.nms.iou_threshold=iou_threshold
        cfg.test_pipeline[1]['img_scale'] = (1024, 1024)
    if mmdet_model in mmdet_models_1024:
        cfg_path = f'/kaggle/working/1024{Path(baseline_cfg_path).name}'
        print(cfg_path)
    else:
        cfg_path = f'/kaggle/working/{Path(baseline_cfg_path).name}'
        print(cfg_path)

    # Save config file for inference later
    cfg.dump(cfg_path)
    #print(f'Config:\n{cfg.pretty_text}')    

In [ ]:
imgs_path = f'/kaggle/tmp/{split}/image/'
test_paths = f'/kaggle/tmp/{split}/image/' + test_df['id'] +'.png'
test_imgs = test_paths.tolist()
for mmdet_model in mmdet_models:
    print("Loading weights from:", models_checkpoint[mmdet_model])
    if mmdet_model in mmdet_models_1024:
        cfg_path = f'/kaggle/working/1024{Path(models_config[mmdet_model]).name}'
    else:
        cfg_path = f'/kaggle/working/{Path(models_config[mmdet_model]).name}'
    cfg = Config.fromfile(cfg_path)
    model = init_detector(cfg, models_checkpoint[mmdet_model], device='cuda:0')
    
    b_mm_list, s_mm_list, l_mm_list = [],[],[]
    
    if mmdet_model in mmdet_models_1024:
        test_dir = f'/kaggle/tmp/{split}/image1024'
        rescal_size = 1024
    else:
        test_dir = f'/kaggle/tmp/{split}/image'
        rescal_size = 640
    for img in os.listdir(test_dir):
        img_id = img.replace('.png', '')
        img_path = f'{test_dir}/{img}'
        imgs = cv2.imread(img_path)
        #print(imgs.shape)
        results = inference_detector(model, imgs)
        result = results[1]
        #print(results[0])
        results_filtered = result[result[:, 4]>0.001]
        bboxes = (results_filtered[:, :4] / rescal_size).tolist()
        scores = results_filtered[:, 4].tolist()
        b_mm_list.append(bboxes)
        s_mm_list.append(scores)
        l_mm_list.append([0] * len(bboxes))
        
    boxes_list.append(b_mm_list)
    scores_list.append(s_mm_list)
    labels_list.append(l_mm_list)

In [ ]:
weights = [2,1,2,1,2,3,2,1,1,3,2]
# weights_final = [5,4,3,2]

iou_thr = 0.5 #  0.5 0.55 0.25
skip_box_thr = 0.001 # 0.1
sigma = 0.1
        
image_ids = []
PredictionStrings = []
num=0
for img in os.listdir(test_dir):
    img_id = img.replace('.png', '')
    image_ids.append(img_id)
    boxes_l, scores_l, labels_l = [], [], []
    for i in range(len(weights)):
        boxes_l.append(boxes_list[i][num])
        scores_l.append(scores_list[i][num])
        labels_l.append(labels_list[i][num])
    num+=1
    
    boxes, scores, labels = weighted_boxes_fusion(boxes_l, scores_l, labels_l, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)  
    
    pre_str = ''
    if len(boxes) > 0:
        w, h = test_df.loc[test_df.id==img_id,['dim1', 'dim0']].values[0]
        bboxes = scale_coords(h, w, boxes)
        for i in range(len(bboxes)):
            pre_str = pre_str + 'opacity %.3f %.0f %.0f %.0f %.0f ' %(scores[i], bboxes[i][0], bboxes[i][1], bboxes[i][2], bboxes[i][3])

    PredictionStrings.append(pre_str.rstrip())

pred_df = pd.DataFrame({'id':image_ids,
                        'PredictionString':PredictionStrings})

In [ ]:
test_df = test_df.drop(['PredictionString'], axis=1)
sub_df = pd.merge(test_df, pred_df, on = 'id', how = 'left').fillna("none 1 0 0 1 1")
sub_df = sub_df[['id', 'PredictionString']]
sub_df['none'] = df_2class['none'] 
for i in range(sub_df.shape[0]):
    if sub_df.loc[i,'PredictionString'] != 'none 1 0 0 1 1':
        sub_df.loc[i,'PredictionString'] = sub_df.loc[i,'PredictionString'] + ' none ' + str(sub_df.loc[i,'none']) + ' 0 0 1 1'
sub_df = sub_df[['id', 'PredictionString']]   
#df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
#df_study = df[:study_len]
df_study = df_study[:study_len]
df_study = df_study.append(sub_df).reset_index(drop=True)
df_study.to_csv('/kaggle/working/submission.csv',index = False)
#sub_df.to_csv('/kaggle/working/submission.csv',index = False)
#df_study